In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error as MSE, mean_absolute_error as MAE
import random


In [54]:

# Фиксируем датчики случайных чисел
random.seed(42)
np.random.seed(42)

# Загружаем данные
df = pd.read_csv('data.csv')

# Разделяем ИСХОДНЫЕ ДАННЫЕ на тренировочную и тестовую выборки
df_train, df_test = train_test_split(df, train_size=0.75, random_state=42)

In [55]:
pd.set_option('display.max_columns', None)

In [56]:

# Класс для удаления столбцов из датасета
class columnDropperTransformer():
    def __init__(self,columns):
        self.columns=columns
    def transform(self,X,y=None):
        X = X.drop(self.columns,axis=1)
        return X
    def fit(self, X, y=None):
        return self

In [57]:
# Класс для обработки типа автомобиля
class columnCarTypeTransformer():
    def __init__(self,columns):
        self.columns=columns
    def transform(self,X,y=None):
        def mod_car_type(x):
            car_type_short = [
                'Внедорожник 3 дв.',
                'Внедорожник 5 дв.',
                'Внедорожник открытый',
                'Кабриолет',
                'Компактвэн',
                'Купе',
                'Лифтбек',
                'Микровэн',
                'Минивэн',
                'Пикап Двойная кабина',
                'Пикап Одинарная кабина',
                'Пикап Полуторная кабина',
                'Родстер',
                'Седан',
                'Спидстер',
                'Тарга',
                'Универсал 5 дв.',
                'Фастбек',
                'Фургон',
                'Хэтчбек 3 дв.',
                'Хэтчбек 4 дв.',
                'Хэтчбек 5 дв.'
            ]
            for car_type in car_type_short:
                if car_type in x:
                    x = car_type
            return x
        X['car_type'] = X['car_type'].apply(lambda x: mod_car_type(x) if not pd.isnull(x) else np.nan)
        return X
    def fit(self, X, y=None):
        return self

In [58]:
# Готовим данные для обработки
drop_columns = ['Unnamed: 0', 'url_car', 'ann_id', 'ann_date', 'ann_city', 
                    'avail', 'original_pts', 'customs', 'link_cpl', 'eng_power_kw', 
                    'pow_resrv', 'options', 'condition', 'url_compl', 'gross_weight']

column_dropper= Pipeline([
    ("column_dropper", columnDropperTransformer(drop_columns))
])

car_type_transformer = Pipeline([
    ("car_type_transformer", columnCarTypeTransformer('car_type'))
])

df_train = column_dropper.fit_transform(df_train)
car_type_transformer.fit_transform(df_train)
df_test = column_dropper.fit_transform(df_test)
car_type_transformer.fit_transform(df_test)

,car_make,car_model,car_gen,car_type,car_compl,car_price,year,mileage,color,eng_size,eng_power,eng_type,transmission,drive,st_wheel,count_owner,state_mark,class_auto,door_count,seat_count,long,widht,height,clearence,v_bag,v_tank,curb_weight,front_brakes,rear_brakes,max_speed,acceleration,fuel_cons,fuel_brand,engine_loc1,engine_loc2,turbocharg,max_torq,cyl_count
73871,Renault Samsung,QM6,I Рестайлинг,Внедорожник 5 дв.,2.0 CVT (144 л.с.),1900000,2021,51000,белый,2.0,144,Бензин,вариатор,передний,Левый,1 владелец,Южная Корея,D,5,5,4675,1845,1680,210.0,NaN,NaN,NaN,дисковые,дисковые,NaN,NaN,7.6,АИ-95,переднее,поперечное,нет,200.0,4.0
93925,Volkswagen,Passat,B7,Седан,1.8 AMT (152 л.с.),1100000,2013,250000,белый,1.8,152,Бензин,роботизированная,передний,Левый,3 или более,Германия,D,4,5,4769,1820,1470,155.0,565.0,70.0,1514.0,дисковые,дисковые,214.0,8.8,5.6,АИ-95,переднее,поперечное,турбонаддув,250.0,4.0
19211,Daewoo,Matiz,I Рестайлинг,Хэтчбек 5 дв.,0.8 MT (52 л.с.),140000,2004,128000,синий,0.8,52,Бензин,механическая,передний,Левый,2 владельца,Южная Корея,A,5,5,3497,1495,1485,150.0,155.0,35.0,770.0,дисковые,барабанные,144.0,17.0,6.3,АИ-92,переднее,поперечное,нет,69.0,3.0
91729,Lada (ВАЗ),2107,1982-2012,Седан,1.5 MT (72 л.с.),65000,2006,700000,белый,1.5,72,Бензин,механическая,задний,Левый,3 или более,Россия,B,4,5,4145,1620,1446,170.0,379.0,39.0,1060.0,дисковые,барабанные,150.0,17.0,7.4,АИ-92,переднее,продольное,нет,105.0,4.0
50436,Mercedes-Benz,GLK-Класс,I (X204) Рестайлинг,Внедорожник 5 дв.,250 2.0 AT (211 л.с.) 4WD,1768000,2014,202167,белый,2.0,211,Бензин,автоматическая,полный,Левый,3 или более,Германия,D,5,5,4536,1840,1669,200.0,450.0,66.0,1825.0,дисковые,дисковые,215.0,7.9,6.8,АИ-95,переднее,продольное,турбонаддув,350.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41428,Land Rover,Discovery,IV,Внедорожник 5 дв.,3.0d AT (249 л.с.) 4WD,2300000,2013,245000,коричневый,3.0,249,Дизель,автоматическая,полный,Левый,2 владельца,Великобритания,J,5,5,4829,1915,1887,185.0,1260.0,82.0,2583.0,дисковые,дисковые,180.0,9.6,8.3,ДТ,переднее,продольное,турбонаддув,600.0,6.0
72034,Renault,Dokker,I,Компактвэн,1.6 MT (82 л.с.),1495000,2018,35000,серебристый,1.6,82,Бензин,механическая,передний,Левый,2 владельца,Франция,M,5,5,4363,1751,1852,190.0,800.0,50.0,1278.0,дисковые,барабанные,159.0,14.3,6.4,АИ-95,переднее,поперечное,нет,134.0,4.0
22585,Datsun,mi-DO,2015-2020,Хэтчбек 5 дв.,1.6 MT (87 л.с.),555000,2017,124474,серебристый,1.6,87,Бензин,механическая,передний,Левый,1 владелец,Япония,B,5,5,3950,1700,1500,174.0,240.0,50.0,1160.0,дисковые,барабанные,170.0,12.0,5.8,АИ-95,переднее,поперечное,нет,140.0,4.0
53549,Mini,Countryman,III,Внедорожник 5 дв.,C 1.5 AMT (170 л.с.),7490000,2023,3990,серый,1.5,170,Бензин,роботизированная,передний,Левый,1 владелец,Великобритания,B,5,5,4445,1843,1635,NaN,450.0,54.0,1545.0,дисковые,дисковые,212.0,8.3,NaN,АИ-95,переднее,поперечное,турбонаддув,280.0,3.0


In [59]:
# Класс для замены полей автомобиля определенным значением по фильтру
class filterTransformer():

    def __init__(self, filters, column_fix, column_replace, value_replace):
        self.filters=filters
        self.column_fix=column_fix
        self.column_replace=column_replace
        self.value_replace=value_replace

    def transform(self,X,y=None):

        def replace_con(df, filter, column_fix, column_replace, value_replace):
            '''
            Функция получает на вход датасет и по укзанному
            значению фильтра в фильтруемом поле, заменяет пропуски
            в заменяем поле на значение value_replace
            '''
            return df.apply(lambda row: value_replace if pd.isnull(row[column_replace]) & (row[column_fix] == filter) else row[column_replace], axis=1)
        
        for filter in self.filters:
            X[self.column_replace] = replace_con(X, filter, self.column_fix, self.column_replace, self.value_replace)
        return X
    
    def fit(self, X, y=None):
        return self

In [60]:
# Создаемдатасеты для замены значений медианами и модами
def df_median_mode(df,columns):
    '''
    Функция получает на вход датасет и формирует по нему датасет
    с медианами и модами расчитанными в группировке
    по полям в списке columns
    '''
    columns_isnumber = df.select_dtypes([int, float]).columns.to_list()
    df_med_mod = df.copy()

    for column in columns_isnumber:
        df_med_mod[column] = df.groupby(columns)[column].transform('median')

    columns_isobject = df.select_dtypes([object]).columns.to_list()

    for column in columns:
        columns_isobject.remove(column)

    for column in columns_isobject:
        df_med_mod[column] = df.groupby(columns)[column].transform(lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan)

    df_med_mod = df_med_mod.drop_duplicates(keep='first')

    return df_med_mod

# медианы и моды в группировке по марке, модели, поколению и типу двигателя
columns = ['car_make', 'car_model', 'car_gen', 'eng_type']
df_mmge = [df_median_mode(df_train, columns).sort_values(by=columns), columns]

# медианы и моды в группировке по марке, модели и поколению
columns = ['car_make', 'car_model', 'car_gen']
df_mmg = [df_median_mode(df_train, columns).sort_values(by=columns), columns]

# медианы и моды в группировке по марке, модели и типу двигателя
columns = ['car_make', 'car_model', 'eng_type']
df_mme = [df_median_mode(df_train, columns).sort_values(by=columns), columns]

# медианы и моды в группировке по марке и модели
columns = ['car_make', 'car_model']
df_mm = [df_median_mode(df_train, columns).sort_values(by=columns), columns]

# медианы и моды в группировке по типу кузова, классу автомобиля и типу двигателя
columns = ['car_type', 'class_auto', 'eng_type']
df_tce = [df_median_mode(df_train, columns).sort_values(by=columns), columns]

# медианы и моды в группировке по типу кузова и классу автомобиля
columns = ['car_type', 'class_auto']
df_tc = [df_median_mode(df_train, columns).sort_values(by=columns), columns]

# медианы и моды в группировке по типу кузова
columns = ['car_type']
df_t = [df_median_mode(df_train, columns).sort_values(by=columns), columns]

# медианы и моды в группировке по классу автомобиля
columns = ['class_auto']
df_c = [df_median_mode(df_train, columns).sort_values(by=columns), columns]

# медианы и моды в группировке по типу кузова и типу двигателя
columns = ['car_type', 'eng_type']
df_te = [df_median_mode(df_train, columns).sort_values(by=columns), columns]

# медианы и моды в группировке по классу автомобиля и типу двигателя
columns = ['class_auto', 'eng_type']
df_ce = [df_median_mode(df_train, columns).sort_values(by=columns), columns]

In [61]:
# Класс для замещения пропуска в поле медианой или модой
class medModTransformer():
    def __init__(self, df_med_mods, column_replace):
        self.df_med_mods = df_med_mods
        self.column_replace = column_replace
    def transform(self, X, y=None):
        def return_med_mod(df_med_mod, row, column_replace):
            '''
            Функция принимает на вход датасет
            с медиананами, модами и полями группировки (до 5 полей),
            значения записи для фильтрации, поле для замены значения
            и возвращает значение поля для указанных параметров
            '''
            df = df_med_mod[0]
            column_group = df_med_mod[1]

            if len(column_group) == 5:
                try:
                    return df[(df[column_group[0]] == row[column_group[0]]) &
                            (df[column_group[1]] == row[column_group[1]]) &
                            (df[column_group[2]] == row[column_group[2]]) &
                            (df[column_group[3]] == row[column_group[3]]) &
                            (df[column_group[4]] == row[column_group[4]])][column_replace].values[0]
                except:
                    return np.nan
            elif len(column_group) == 4:
                try:
                    return df[(df[column_group[0]] == row[column_group[0]]) &
                            (df[column_group[1]] == row[column_group[1]]) &
                            (df[column_group[2]] == row[column_group[2]]) &
                            (df[column_group[3]] == row[column_group[3]])][column_replace].values[0]
                except:
                    return np.nan
            elif len(column_group) == 3:
                try:
                    return df[(df[column_group[0]] == row[column_group[0]]) &
                            (df[column_group[1]] == row[column_group[1]]) &
                            (df[column_group[2]] == row[column_group[2]])][column_replace].values[0]
                except:
                    return np.nan
            elif len(column_group) == 2:
                try:
                    return df[(df[column_group[0]] == row[column_group[0]]) &
                            (df[column_group[1]] == row[column_group[1]])][column_replace].values[0]
                except:
                    return np.nan
            elif len(column_group) == 1:
                try:
                    return df[(df[column_group[0]] == row[column_group[0]])][column_replace].values[0]
                except:
                    return np.nan

        for df_med_mod in self.df_med_mods:
            X[self.column_replace] = X.apply(lambda row: return_med_mod(df_med_mod, row, self.column_replace) if pd.isnull(row[self.column_replace]) else row[self.column_replace], axis=1)
        return X
    def fit(self, X, y=None):
        return self

In [62]:
# Класс для преобразования количества владельцев в число
class classOwnsTransformer():

    def transform(self,X,y=None):
        X['count_owner'] = X['count_owner'].apply(lambda x: int(x.split()[0]) if not pd.isnull(x) else np.nan)
        return X
    
    def fit(self, X, y=None):
        return self

In [63]:
# Класс для удаления дубликатов
class dropDuplicate():

    def transform(self,X,y=None):
        column_features = df_train.columns[df_train.any()].to_list()
        columns_remove = ['car_price', 'car_model', 'car_gen', 'car_compl']
        for column_remove in columns_remove:
            column_features.remove(column_remove)
        X = X.drop_duplicates(subset=column_features, keep='first')
        X = X.reset_index(drop=True)
        return X
    
    def fit(self, X, y=None):
        return self

In [64]:
# Класс для обновления индекса
class resetIndex():

    def transform(self,X,y=None):
        X = X.reset_index(drop=True)
        return X
    
    def fit(self, X, y=None):
        return self

In [65]:
# Создаем преобразователи данных
column_dropper = Pipeline([
    ("column_dropper", columnDropperTransformer(['Unnamed: 0', 'url_car', 'ann_id', 'ann_date', 'ann_city', 
                                                 'avail', 'original_pts', 'customs', 'link_cpl', 'eng_power_kw', 
                                                 'pow_resrv', 'options', 'condition', 'url_compl', 'gross_weight']))
])

car_type_transformer = Pipeline([
    ("car_type_transformer", columnCarTypeTransformer('car_type'))
])

class_auto_transformer = Pipeline(steps=[
    ("transform_class_auto_mmge", medModTransformer([df_mmge, df_mmg, df_mme, df_mm], 'class_auto')),
    ("transform_class_auto_to_M", filterTransformer(['Фургон'], 'car_type', 'class_auto', 'M')),
    # ("transform_class_auto_long", classLongTransformer('car_type', 'long')),
    ("transform_class_auto_tce", medModTransformer([df_tce, df_tc, df_te, df_t, df_ce, df_c], 'class_auto')),
    ("transform_class_auto_to_F", filterTransformer(['Лимузин'], 'car_type', 'class_auto', 'F'))
])

eng_size_transformer= Pipeline([
    ("eng_size_transformer", filterTransformer(['Электро'], 'eng_type', 'eng_size', 0))
])

clearence_transformer= Pipeline([
    ("clearence_transformer", medModTransformer([df_mmge, df_mmg, df_mme, df_mm, df_tce, df_tc, df_te, df_t, df_ce, df_c], 'clearence'))
])

v_bag_transformer = Pipeline(steps=[
    ("transform_v_bag_class_auto_to_0", filterTransformer(['S'], 'class_auto', 'v_bag', 0)),
    ("transform_v_bag_car_type_to_0", filterTransformer(['Кабриолет', 'Купе', 'Пикап Двойная кабина', 'Пикап Одинарная кабина', 
                                                         'Пикап Полуторная кабина', 'Родстер', 'Тарга'], 
                                                         'car_type', 'v_bag', 0)),
    ("transform_v_bag_mmg", medModTransformer([df_mmge, df_mmg, df_mme, df_mm, df_tce, df_tc, df_te, df_t, df_ce, df_c], 'v_bag'))
])

v_tank_transformer = Pipeline(steps=[
    ("transform_v_tank_eng_type_to_0", filterTransformer(['Электро'], 'eng_type', 'v_tank', 0)),
    ("transform_v_tank_mmg", medModTransformer([df_mmge, df_mmg, df_mme, df_mm, df_tce, df_tc, df_te, df_t, df_ce, df_c], 'v_tank'))
])

curb_weight_transformer = Pipeline([
    ("curb_weight_transformer", medModTransformer([df_mmge, df_mmg, df_mme, df_mm, df_tce, df_tc, df_te, df_t, df_ce, df_c], 'curb_weight'))
])

rear_brakes_transformer = Pipeline([
    ("rear_brakes_transformer", medModTransformer([df_mmge, df_mmg, df_mme, df_mm, df_tce, df_tc, df_te, df_t, df_ce, df_c], 'rear_brakes'))
])

max_speed_transformer= Pipeline([
    ("max_speed_transformer", medModTransformer([df_mmge, df_mmg, df_mme, df_mm, df_tce, df_tc, df_te, df_t, df_ce, df_c], 'max_speed'))
])

acceleration_transformer= Pipeline([
    ("acceleration_transformer", medModTransformer([df_mmge, df_mmg, df_mme, df_mm, df_tce, df_tc, df_te, df_t, df_ce, df_c], 'acceleration'))
])

fuel_cons_transformer = Pipeline(steps=[
    ("transform_fuel_cons_eng_type_to_0", filterTransformer(['Электро'], 'eng_type', 'fuel_cons', 0)),
    ("transform_fuel_cons_mmg", medModTransformer([df_mmge, df_mmg, df_mme, df_mm, df_tce, df_tc, df_te, df_t, df_ce, df_c], 'fuel_cons'))
])

fuel_brand_transformer = Pipeline(steps=[
    ("transform_fuel_brand_eng_type_to_0", filterTransformer(['Электро'], 'eng_type', 'fuel_brand', 'Nan')),
    ("transform_fuel_brand_mmg", medModTransformer([df_mmge, df_mmg, df_mme, df_mm, df_tce, df_tc, df_te, df_t, df_ce, df_c], 'fuel_brand'))
])

cyl_count_transformer = Pipeline(steps=[
    ("transform_cyl_count_eng_type_to_0", filterTransformer(['Электро'], 'eng_type', 'cyl_count', 0)),
    ("transform_cyl_count_mmg", medModTransformer([df_mmge, df_mmg, df_mme, df_mm, df_tce, df_tc, df_te, df_t, df_ce, df_c], 'cyl_count'))
])

engine_loc1_transformer = Pipeline(steps=[
    ("transform_engine_loc1_eng_type_to_0", filterTransformer(['Электро'], 'eng_type', 'engine_loc1', 'Nan')),
    ("transform_engine_loc1_mmg", medModTransformer([df_mmge, df_mmg, df_mme, df_mm, df_tce, df_tc, df_te, df_t, df_ce, df_c], 'engine_loc1'))
])

engine_loc2_transformer = Pipeline(steps=[
    ("transform_engine_loc2_eng_type_to_0", filterTransformer(['Электро'], 'eng_type', 'engine_loc2', 'Nan')),
    ("transform_engine_loc2_mmg", medModTransformer([df_mmge, df_mmg, df_mme, df_mm, df_tce, df_tc, df_te, df_t, df_ce, df_c], 'engine_loc2'))
])

turbocharg_transformer = Pipeline(steps=[
    ("transform_turbocharg_eng_type_to_0", filterTransformer(['Электро'], 'eng_type', 'turbocharg', 'Nan')),
    ("transform_turbocharg_mmg", medModTransformer([df_mmge, df_mmg, df_mme, df_mm, df_tce, df_tc, df_te, df_t, df_ce, df_c], 'turbocharg'))
])

max_torq_transformer= Pipeline([
    ("transformer_max_torq", medModTransformer([df_mmge, df_mmg, df_mme, df_mm, df_tce, df_tc, df_te, df_t, df_ce, df_c], 'max_torq'))
])

count_owner_transformer = Pipeline([
    ("count_owner_transformer", classOwnsTransformer())
])

drop_duplicate = Pipeline([
    ("drop_duplicate", dropDuplicate())
])

reset_index = Pipeline([
    ("reset_index", resetIndex())
])


In [66]:
# Объединяем преобразователи для трэйна
column_null_transformer = Pipeline(steps=[
    ('class_auto_transformer', class_auto_transformer),
    ('eng_size_transformer', eng_size_transformer),
    ('clearence_transformer', clearence_transformer),
    ('v_bag_transformer', v_bag_transformer),
    ('v_tank_transformer', v_tank_transformer),
    ('curb_weight_transformer', curb_weight_transformer),
    ('rear_brakes_transformer', rear_brakes_transformer),
    ('max_speed_transformer', max_speed_transformer),
    ('acceleration_transformer', acceleration_transformer),
    ('fuel_cons_transformer', fuel_cons_transformer),
    ('fuel_brand_transformer', fuel_brand_transformer),
    ('cyl_count_transformer', cyl_count_transformer),
    ('engine_loc1_transformer', engine_loc1_transformer),
    ('engine_loc2_transformer', engine_loc2_transformer),
    ('turbocharg_transformer', turbocharg_transformer),
    ('max_torq_transformer', max_torq_transformer),
    ("column_dropper", columnDropperTransformer(['car_model', 'car_gen', 'car_compl'])),
    ("count_owner_transformer", count_owner_transformer),
    ("drop_duplicate", drop_duplicate)
])

In [67]:
# Объединяем преобразователи для теста
column_null_transformer_test = Pipeline(steps=[
    ('class_auto_transformer', class_auto_transformer),
    ('eng_size_transformer', eng_size_transformer),
    ('clearence_transformer', clearence_transformer),
    ('v_bag_transformer', v_bag_transformer),
    ('v_tank_transformer', v_tank_transformer),
    ('curb_weight_transformer', curb_weight_transformer),
    ('rear_brakes_transformer', rear_brakes_transformer),
    ('max_speed_transformer', max_speed_transformer),
    ('acceleration_transformer', acceleration_transformer),
    ('fuel_cons_transformer', fuel_cons_transformer),
    ('fuel_brand_transformer', fuel_brand_transformer),
    ('cyl_count_transformer', cyl_count_transformer),
    ('engine_loc1_transformer', engine_loc1_transformer),
    ('engine_loc2_transformer', engine_loc2_transformer),
    ('turbocharg_transformer', turbocharg_transformer),
    ('max_torq_transformer', max_torq_transformer),
    ("column_dropper", columnDropperTransformer(['car_model', 'car_gen', 'car_compl'])),
    ("count_owner_transformer", count_owner_transformer),
    ("reset_index", reset_index)
])

In [68]:
# Преобразуем и разделяем признаки и таргет
train = column_null_transformer.fit_transform(df_train)
test = column_null_transformer_test.fit_transform(df_test)
X_train = train.copy()
y_train = np.log(train['car_price'])
X_test = test.copy()
y_test = np.log(test['car_price'])
X_train = X_train.drop(['car_price'], axis=1)
X_test = X_test.drop(['car_price'], axis=1)

In [69]:
# Собственно, наш пайплайн
cat_features_mask = (X_train.dtypes == "object").values

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first', min_frequency=4, max_categories=140, handle_unknown='ignore'))  # OHE-кодирование
])

numerical_transformer = Pipeline(steps=[
    ('poly', PolynomialFeatures(degree=2)), # Poly-кодирование
    ('scaler', StandardScaler()) # Стандартизируем
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, X_train.columns[cat_features_mask]), # Обрабатываем категориальные признаки
        ('num', numerical_transformer, X_train.columns[~cat_features_mask]) # Обрабатываем числовые признаки
    ]
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge(alpha=8.3)) # Используем Ridge
])

pipeline.fit(X_train, y_train) # Обучаем модель

y_pred = pipeline.predict(X_test) # Создаем переменные для вывода результатов
y_pred_train = pipeline.predict(X_train)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [70]:
# Функция вывода основных характеристик модели (ориентируемся прежде всего на R^2)
def print_r2_mse(y_train, y_test, y_pred):
    '''
    Функция принимает на вход признаки и таргет для тренировочных
    и тестовых данных, модель и выводит на экран 
    значения MSE, R^2 для трейна и теста для указанных параметров
    '''
    print('*'*20)
    print(f'Значение MAE для трейна:  {MAE(np.exp(y_train), np.exp(y_pred_train))}')
    print(f'Значение MAE для теста:   {MAE(np.exp(y_test), np.exp(y_pred))}')
    print('*'*20)
    print(f'Значение MSE для трейна:  {MSE(np.exp(y_train), np.exp(y_pred_train))}')
    print(f'Значение MSE для теста:   {MSE(np.exp(y_test), np.exp(y_pred))}')
    print('*'*20)
    print(f'Значение R^2 для трейна:  {r2_score(y_train, y_pred_train)}')
    print(f'Значение R^2 для теста:   {r2_score(y_test, y_pred)}')

In [71]:
print_r2_mse(y_train, y_test, y_pred) # Выводим результаты, довольны собой

********************
Значение MAE для трейна:  511201.4706406493
Значение MAE для теста:   523356.5367841127
********************
Значение MSE для трейна:  6500084651059.389
Значение MSE для теста:   13256581088747.7
********************
Значение R^2 для трейна:  0.9191456358574627
Значение R^2 для теста:   0.9196838140346884
